In [81]:
import numpy as np
import pandas as pd
import cv2
import os
import subprocess
from pathlib import Path

In [5]:
# Load Annotations
annotations = pd.read_csv('/Users/Samuel/Documents/EPFL/S2/Deep Learning/Project/data/raw/videos/HateMM_annotation.csv')
annotations

,video_file_name,label,hate_snippet,target
0,hate_video_1.mp4,Hate,"[['00:00:34', '00:01:34']]",Blacks
1,hate_video_2.mp4,Hate,"[['00:00:06', '00:02:06']]",Blacks
2,non_hate_video_1.mp4,Non Hate,NaN,Others
3,hate_video_3.mp4,Hate,"[['00:00:03', '00:01:40'], ['00:01:41', '00:03...",Blacks
4,non_hate_video_2.mp4,Non Hate,NaN,Blacks
...,...,...,...,...
1078,non_hate_video_650.mp4,Non Hate,NaN,['Whites']
1079,hate_video_430.mp4,Hate,"[['00:00:09', '00:04:04']]",['Jews']
1080,non_hate_video_651.mp4,Non Hate,NaN,['Others']
1081,non_hate_video_652.mp4,Non Hate,NaN,['Others']


In [30]:
# Load video
video_path = '/Users/Samuel/Documents/EPFL/S2/Deep Learning/Project/data/raw/videos/hateful/hate_video_3.mp4'
video_name = video_path.split('/')[-1]
video_hate_snippets = eval(annotations[annotations['video_file_name'] == video_name]['hate_snippet'].values[0])
video_hate_snippets = [[sum(x*int(t) for x, t in zip([3600, 60, 1], time.split(':'))) for time in hate_snippets] for hate_snippets in video_hate_snippets]

# Check if snippets are less than 5 seconds appart then concatenate
video_hate_snippet = []
for i in range(len(video_hate_snippets)):
    if i == 0:
        video_hate_snippet.append(video_hate_snippets[i])
    else:
        if video_hate_snippets[i][0] - video_hate_snippets[i-1][1] < 5:
            video_hate_snippet[-1][1] = video_hate_snippets[i][1]
        else:
            video_hate_snippet.append(video_hate_snippets[i])
print(video_hate_snippet)

# Extract Hate Snippet 
video = cv2.VideoCapture(video_path)
print(type(video))
fps = video.get(cv2.CAP_PROP_FPS)
start_frame = int(video_hate_snippet[0] * fps)
end_frame = int(video_hate_snippet[1] * fps)
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Create directory to save video snippet as mp4
output_dir = '/Users/Samuel/Documents/EPFL/S2/Deep Learning/Project/data/clean/videos/hateful/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = os.path.join(output_dir, video_name)
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (int(video.get(3)), int(video.get(4))))

# Read and save frames
while True:
    ret, frame = video.read()
    if not ret:
        break
    current_frame = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    if current_frame > end_frame:
        break
    video_writer.write(frame)
# Release video objects
video.release()
video_writer.release()

[[3, 207]]
<class 'cv2.VideoCapture'>


TypeError: can't multiply sequence by non-int of type 'float'

In [84]:
# def extract_hate_snippet(video_path: str, start_frame: int, end_frame: int, output_path: str):
#     """
#     Extracts a snippet from a video and saves it to the specified output path.
    
#     Parameters:
#     - video: cv2.VideoCapture object
#     - start_frame: int, starting frame number
#     - end_frame: int, ending frame number
#     - output_path: str, path to save the output video
#     """
#     video = cv2.VideoCapture(video_path)

#     # Create video writer
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     fps = video.get(cv2.CAP_PROP_FPS)
#     video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
#     video_writer = cv2.VideoWriter(output_path, fourcc, fps, (int(video.get(3)), int(video.get(4))))
    
#     # Read and save frames
#     while True:
#         ret, frame = video.read()
#         if not ret:
#             break
#         current_frame = int(video.get(cv2.CAP_PROP_POS_FRAMES))
#         if current_frame > end_frame:
#             break
#         video_writer.write(frame)
    
#     # Release video objects
#     video.release()
#     video_writer.release()

def extract_hate_snippet(video_path: Path, start: float, end: float, output_path: Path):
    """
    Extracts a snippet from a video and saves it to the specified output path.
    
    Parameters:
    - video_path: str, path to the video file
    - start: float, start time in seconds
    - end: float, end time in seconds
    - output_path: str, path to save the output video
    """
    cmd = [
        "ffmpeg",
        "-ss", start,
        "-to", end,
        "-i", str(video_path),
        "-c:v", "copy",
        "-c:a", "copy",
        "-y",
        str(output_path)
    ]
    subprocess.run(cmd, check=True)

def process_video(video_path, annotations, output_dir):
    """
    Creates hate snippets from a video based on the provided annotations.
    
    Parameters:
    - video_path: str, path to the video file
    - annotations: pd.DataFrame, DataFrame containing the annotations
    """

    video_name = os.path.basename(video_path)

    # Get Hateful Snippets
    video_hate_snippets = eval(annotations[annotations['video_file_name'] == video_name]['hate_snippet'].values[0])
    print(video_hate_snippets)
    video_hate_snippets_seconds = [[sum(x*int(t) for x, t in zip([3600, 60, 1], time.split(':'))) for time in hate_snippets] for hate_snippets in video_hate_snippets] # Convert to seconds

    # Check if snippets are less than 5 seconds appart then concatenate, format 00:00:00
    video_hate_snippets_concat = []
    for i in range(len(video_hate_snippets)):
        if i == 0:
            video_hate_snippets_concat.append(video_hate_snippets[i])
        else:
            if video_hate_snippets_seconds[i][0] - video_hate_snippets_seconds[i-1][1] < 5:
                video_hate_snippets_concat[-1][1] = video_hate_snippets[i][1]
            else:
                video_hate_snippets_concat.append(video_hate_snippets[i])
        
    print(video_hate_snippets_concat)
    
    # Create directory to save video snippet as mp4
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Extract each hate snippet
    for i in range(len(video_hate_snippets)):
        start = video_hate_snippets[i][0]
        end = video_hate_snippets[i][1]
        output_path = os.path.join(output_dir, f"{video_name.split('.')[0]}_snippet_{i}.mp4")
        extract_hate_snippet(video_path, start, end, output_path)

In [ ]:
# Test code

video_paths = [
    '/Users/Samuel/Documents/EPFL/S2/Deep Learning/Project/data/raw/videos/hateful/hate_video_29.mp4'
]
output_dir = '/Users/Samuel/Documents/EPFL/S2/Deep Learning/Project/data/clean/videos/hateful/'
for video_path in video_paths:
    process_video(video_path, annotations, output_dir)

[['00:00:34', '00:02:31'], ['00:03:07', '00:03:44'], ['00:04:04', '00:04:50'], ['00:05:11', '00:05:58'], ['00:06:14', '00:06:54'], ['00:07:08', '00:08:01'], ['00:09:45', '00:10:18']]
[['00:00:34', '00:02:31'], ['00:03:07', '00:03:44'], ['00:04:04', '00:04:50'], ['00:05:11', '00:05:58'], ['00:06:14', '00:06:54'], ['00:07:08', '00:08:01'], ['00:09:45', '00:10:18']]


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex